In [29]:
import keras

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import os
import numpy as np
import cv2
import random

tf.logging.set_verbosity(tf.logging.INFO)

# emotion coding: 0=neutral, 1=anger, 2=contempt, 3=disgust, 4=fear, 5=happy, 6=sadness, 7=surprise
# e_labels = [0, 1, 2, 3, 4, 5, 6, 7]
# this is not in the domain of our loss function, so we will increase all values by 1

ModuleNotFoundError: No module named 'sklearn'

In [34]:
def get_tagged_images():
    emotions = []
    for path, name, files in os.walk('Emotion'):
        for file in files:
            if(file):
                with open(os.path.join(path, file)) as curf:
                    emotions.append((os.path.join(path, file), file[:-4], int(curf.read().strip(' ')[:1]) + 1))

    print(emotions[0])
    
    img = os.path.join('aligned', emotions[0][1][:-8] + '.png')
    im = (cv2.imread(img, 0))
    nim = cv2.normalize(im.astype('float'), None, 0, 1,norm_type=cv2.NORM_MINMAX)

    print(len(emotions))
    return emotions

emotions = get_tagged_images()

('Emotion\\S005\\001\\S005_001_00000011_emotion.txt', 'S005_001_00000011_emotion', 4)
327


In [42]:
# want shape (num images, 115,600)

def get_image_data():
    
    data = []
    labels = []

    for i in range(len(emotions)):        
        im = os.path.join(os.path.join('aligned', emotions[i][1][:-8] + '.png'))
        nimg = cv2.normalize(cv2.imread(im, 0).astype('float'), None, 0, 1,norm_type=cv2.NORM_MINMAX)

        data.append(nimg.flatten())
        labels.append(emotions[i][2])

    npdata = np.array(data, dtype=np.float32)
    nplabels = np.array(labels, dtype=np.int32)

    assert not np.any(np.isnan(npdata))
    assert not np.any(np.isnan(nplabels))

    print(npdata.shape)
    npdata
    return npdata, nplabels
    
a, b = get_image_data()
print(a[0].shape)

(327, 115600)
(115600,)


In [44]:
m = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=[340, 340, 3], pooling=max)

m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 340, 340, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 340, 340, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 340, 340, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 170, 170, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 170, 170, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 170, 170, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 85, 85, 128)       0         
__________